In [1]:
import pandas as pd
import numpy as np
import json

### Extracting Data

In [2]:
import csv

In [3]:
data_path = '../../twitter-swisscom/twex.tsv'

In [4]:
columns_names = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId', 'inReplyTo', 'source',
                 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl', 'userName', 'screenName',
                 'followersCount', 'friendsCount', 'statusesCount', 'userLocation']

# the columns that interest us for the density map
columns_to_keep = ['id', 'createdAt', 'placeLatitude', 'placeLongitude']

In [5]:
# set to None to get all the records
num_rows = None

In [6]:
df_data = pd.read_csv(data_path, sep='\t', encoding='utf-8', escapechar='\\', quoting=csv.QUOTE_NONE,
                      header=None, na_values='N', nrows=num_rows, names=columns_names)

# give labels to the columns
df_data.columns = columns_names

C:\Users\Inês Valentim\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,1,2,3,5,6,8,9,10,11,12,13,14,15,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_data.dtypes

id                 object
userId             object
createdAt          object
text               object
longitude         float64
latitude           object
placeId            object
inReplyTo         float64
source             object
truncated          object
placeLatitude      object
placeLongitude     object
sourceName         object
sourceUrl          object
userName           object
screenName         object
followersCount     object
friendsCount      float64
statusesCount     float64
userLocation       object
dtype: object

In [8]:
df_data = df_data[columns_to_keep]

In [9]:
# number of tweets before dropping NaNs
df_data.shape

(28799234, 4)

In [10]:
# make sure the ID of the tweets is a number
df_data['numeric_id'] = df_data['id'].apply(pd.to_numeric, errors='coerce')

In [11]:
# make sure all tweets are geolocated and have an ID and a timestamp associated
df_data.dropna(inplace=True)
df_data.drop('numeric_id', axis='columns', inplace=True)

In [12]:
# number of tweets after dropping NaNs
df_data.shape

(20189524, 4)

### Parsing Date

In [13]:
import numpy as np
from datetime import datetime

In [14]:
# according to the schema, dates should have the format 0000-00-00 00:00:00
df_data = df_data[df_data['createdAt'].str.len() == 19]
df_data['datetime'] = pd.to_datetime(df_data['createdAt'], errors='coerce')
df_data.dropna(inplace=True)

In [15]:
df_data['year'] = pd.DatetimeIndex(df_data['datetime']).year
df_data['month'] = pd.DatetimeIndex(df_data['datetime']).month

In [16]:
# only consider tweets from 2010 or later
df_data = df_data[df_data['year'] >= 2010]

In [17]:
# just to get an idea of the distribution of tweets per year
df_data['year'].value_counts()

2014    5791960
2013    5211698
2015    3508696
2016    2818604
2012    2757776
2011      83022
2010      17733
Name: year, dtype: int64

In [18]:
df_data.shape

(20189489, 7)

In [19]:
df_data['dates'] = list(zip(df_data['month'], df_data['year']))

In [20]:
series_dates = pd.Series(df_data['dates'].unique())
unix_times = pd.DataFrame({'dates': series_dates})

In [21]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['dates'][1], record['dates'][0], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [22]:
unix_times['unix_time'] = unix_times.apply(convert_to_unix_time, axis=1)

In [23]:
df_data_dates = pd.merge(df_data, unix_times, how='inner', on='dates')

In [24]:
df_data_dates.drop(['createdAt', 'datetime', 'year', 'month', 'dates'], axis=1, inplace=True)

In [25]:
df_data_dates.shape

(20189489, 4)

In [26]:
df_data_dates.to_csv(path_or_buf='clean_dates.csv', index=False)

### Getting Geolocation

In [2]:
df_data_dates = pd.read_csv('clean_dates.csv')

In [3]:
df_data_dates.shape

(20189489, 4)

In [4]:
from shapely.geometry import Point, shape

In [5]:
df_data_dates['coords'] = list(zip(df_data_dates['placeLongitude'], df_data_dates['placeLatitude']))

In [6]:
# create a DataFrame with the unique pairs of coordinates
series_coords = pd.Series(df_data_dates['coords'].unique())
coords_cantons = pd.DataFrame({'coords': series_coords})

In [7]:
coords_cantons.shape

(16413, 1)

In [8]:
# get the ID of cantons or municipalities, depending on the geoJSON file passed as an argument
def get_id_from_coords(record, filename):
    point = Point(float(record['coords'][0]), float(record['coords'][1]))
    
    for feature in filename['features']:
        if(feature['geometry']['type'] == 'Polygon'):
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                return feature['properties']['id']
        elif(feature['geometry']['type'] == 'MultiPolygon'):
            multipolygon = shape(feature['geometry'])
            for polygon in multipolygon:
                if polygon.contains(point):
                    return feature['properties']['id']

In [9]:
# read geoJSON file with canton names and IDs
path_cantons = '../res/topo/ch-cantons-geo.json'

with open(path_cantons) as file:
    cantons_json = json.load(file)

In [10]:
# for each pair of coordinates, get the ID corresponding to the canton
coords_cantons['canton_id'] = coords_cantons.apply(get_id_from_coords, args=(cantons_json,), axis=1)

In [11]:
print('number of coordinates outside CH:', coords_cantons['canton_id'].isnull().sum())

number of coordinates outside CH: 6827


In [12]:
# drop pairs of coordinates that are located outside of CH
coords_cantons.dropna(inplace=True)

In [13]:
coords_cantons.shape

(9586, 2)

In [14]:
# merge the data based on the coordinates
df_data_cantons = pd.merge(df_data_dates, coords_cantons, how='inner', on='coords')

In [15]:
df_data_cantons.shape

(12705241, 6)

In [16]:
df_data_cantons.to_csv(path_or_buf='clean_cantons.csv', index=False)

In [20]:
# read geoJSON file with municipalities names and IDs
path_towns = '../res/topo/ch-municipalities-geo.json'

with open(path_towns) as file:
    towns_json = json.load(file)

In [21]:
# for each pair of coordinates, get the ID corresponding to the municipality
coords_cantons['town_id'] = coords_cantons.apply(get_id_from_coords, args=(towns_json,), axis=1)

In [22]:
coords_cantons.shape

(9586, 3)

In [24]:
# merge the data based on the coordinates
df_data_towns = pd.merge(df_data_cantons, coords_cantons, how='inner', on='coords')

In [27]:
df_data_towns.shape

(12705241, 8)

In [28]:
df_data_towns.to_csv(path_or_buf='clean_towns.csv', index=False)

### Grouping Tweets by Year and Canton/Municipality

In [17]:
grouped_year_canton = df_data_cantons.groupby(['unix_time', 'canton_id']).size()

In [29]:
grouped_year_town = df_data_towns.groupby(['unix_time', 'town_id']).size()

### Creating JSON Files

In [18]:
def create_json_file(canton_municipality, grouped_dataframe, output_filename):
    dates_list = list(grouped_dataframe.index.levels[0])
    
    if canton_municipality == 'c':
        main_object = 'cantons'
    else:
        main_object = 'municipalities'
    
    json_file = dict()
    json_file[main_object] = list()  
    
    for date_index in range(len(dates_list)):
        json_file[main_object].append(dict())
        json_file[main_object][date_index]['date'] = int(dates_list[date_index])
        json_file[main_object][date_index]['data'] = list()
        
        ids_list = list(grouped_dataframe[dates_list[date_index]].index)

        for id_index in range(len(ids_list)):
            json_file[main_object][date_index]['data'].append(dict())
            json_file[main_object][date_index]['data'][id_index]['id'] = int(ids_list[id_index])
            json_file[main_object][date_index]['data'][id_index]['nbr'] = int(grouped_dataframe[(dates_list[date_index],
                                                                                                 ids_list[id_index])])
            
    with open(output_filename, 'w') as file:
        json.dump(json_file, file)

In [19]:
create_json_file('c', grouped_year_canton, '../res/density/canton_density.json')

In [30]:
create_json_file('m', grouped_year_town, '../res/density/municipality_density.json')